HR分析をします。

離職は経営に様々な影響を与えるので、経営陣の組織変革、社員のリテンションのために、疲弊状況を可視化する。（ロジスティック回帰で）


・説明
XYZという名前の大企業は、ある時点で約4000人の従業員を雇用しています。ただし、毎年、従業員の約15％が退職し、求人市場で利用可能な人材プールに置き換える必要があります。経営陣は、次の理由により、このレベルの減少（従業員が自分で、または解雇されたために退職する）は会社にとって悪いと考えています。

元従業員のプロジェクトが遅れ、タイムラインを満たすことが困難になり、消費者やパートナーの間で評判が失われる
新しい人材を採用するために、かなりの部門を維持する必要があります。
多くの場合、新入社員は仕事について訓練を受けるか、会社に慣れるための時間を与える必要があります
したがって、経営陣は、人為的な損耗を抑制するためにどの要素に焦点を当てるべきかを理解するよう人事分析会社に委託しています。つまり、従業員のほとんどがリテンションする為に、職場にどのような変更を加える必要があるかを知りたいのです。また、これらの変数のどれが最も重要であり、すぐに対処する必要があるかを知りたいと考えています。

あなたは会社のスターアナリストの1人なので、このプロジェクトはあなたに与えられました。

ケーススタディの目的
ロジスティック回帰を使用して、社員の疲弊状況の確率をモデル化する必要があります。このようにして得られた結果は、経営陣が従業員のほとんどをリテンションさせるために、職場にどのような変更を加える必要があるかを理解するために使用されます。

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [ ]:
# 説明のファイル
dic = pd.read_excel('../input/hr-analytics-case-study/data_dictionary.xlsx')
dic

* Attrition　前年に退職したかどうか
* BusinessTravel　出張頻度
* Department　部門
* DistanceFromHome　通勤距離
* EducationField　選考
* MaritalStatus　婚姻状況
* NumCompaniesWorked　在籍社数
* percent salary hike 給与UP率
* YearsSinceLastPromotion　最後の昇級からの年数
* YearsWithCurrManager　現在のマネジャーと一緒の年数

In [ ]:
# ファイル読み込んでいく
employee = pd.read_csv('../input/hr-analytics-case-study/employee_survey_data.csv')
general = pd.read_csv('../input/hr-analytics-case-study/general_data.csv')
intime = pd.read_csv('../input/hr-analytics-case-study/in_time.csv')
outtime = pd.read_csv('../input/hr-analytics-case-study/out_time.csv')
mgrsurvey = pd.read_csv('../input/hr-analytics-case-study/manager_survey_data.csv')

In [ ]:
employee.head()
# 満足度とWLBの指標

In [ ]:
general.head()
# 全体的なデータ
# IDでマージできそう

In [ ]:
intime.head()
# 勤怠の出社時間

In [ ]:
outtime.head()
# 勤怠の退社時間

In [ ]:
mgrsurvey.head()
# 仕事への関与とパフォーマンスの評価

generalのAttrition「前年退職したかどうか」が目的変数

* intimeとouttime計算して労働時間をみる

* generalとemployeeとmgrsurveyを合体させてEDAしてみる

In [ ]:
intime = intime.apply(pd.to_datetime)
outtime = outtime.apply(pd.to_datetime)

time = outtime - intime
time.head()

In [ ]:
time = time/ np.timedelta64(1, 's')
time.head()

In [ ]:
intime = intime.applymap(lambda x: 3600*x.hour + 60*x.minute + x.second)
outtime = outtime.applymap(lambda x: 3600*x.hour + 60*x.minute + x.second)

In [ ]:
general['in_mean'] = intime.mean(axis=1)
general['out_mean'] = outtime.mean(axis=1)

general['in_std'] = intime.std(axis=1)
general['out_std'] = outtime.std(axis=1)

In [ ]:
general.head()

In [ ]:
# generalに労働時間を入れる
general['time_std'] = general['out_std'] - general['in_std']
general.head()

In [ ]:
general.isnull().sum()

In [ ]:
# 列を強制的に全表示する
pd.set_option('display.max_columns', 100)

In [ ]:
general.head()

In [ ]:
# general, employee, mgrsurveyをEmployeeIDで結合する

df1 = pd.merge(general, employee, on='EmployeeID')

In [ ]:
df = pd.merge(df1, mgrsurvey, on='EmployeeID')

In [ ]:
df.head()

****いろいろとEDAしていく****

* MaritalStatus
* JobRole
* Gender
* EducationField
* Department
* BusinessTravel

質的カテゴリーから

In [ ]:
# 婚姻状況での違い

df_mar = pd.crosstab(df['Attrition'], df['MaritalStatus'])
print(df_mar)

df_mar.plot.bar()
# Singleの離職が多い

In [ ]:
# 役職での違い

df_job = pd.crosstab(df['Attrition'], df['JobRole'])
print(df_job)

df_job.plot.bar()
# あまり大きな関係なさそう

In [ ]:
# 性別での違い

df_gen = pd.crosstab(df['Attrition'], df['Gender'])
print(df_gen)

df_gen.plot.bar()
# 性別は関係なさそう

In [ ]:
# 専攻での違い

df_edu = pd.crosstab(df['Attrition'], df['EducationField'])
print(df_edu)

df_edu.plot.bar()
# 関係なさそう

In [ ]:
# 部署別の退職数

df_dep = pd.crosstab(df['Attrition'], df['Department'])
print(df_dep)

df_dep.plot.bar()
# 比率変わらないのであまり関係ない？

In [ ]:
# 出張の頻度別の退職数

df_biz = pd.crosstab(df['Attrition'], df['BusinessTravel'])
print(df_biz)

df_biz.plot.bar()
# 出張の頻度は関係なさそう

In [ ]:
# 環境への満足度別の退職者数

df_es = pd.crosstab(df['Attrition'], df['EnvironmentSatisfaction'])
print(df_es)
df_es.plot.bar()
# 辞める人は高くても辞めている

In [ ]:
# 仕事への満足度別の退職者数

df_js = pd.crosstab(df['Attrition'], df['JobSatisfaction'])
print(df_js)
df_js.plot.bar()
# 辞める人は高くても辞めている

In [ ]:
# ワークライフバランス別の退職者数

df_wb = pd.crosstab(df['Attrition'], df['WorkLifeBalance'])
print(df_wb)
df_wb.plot.bar()
# 3.0=Betterでも辞めている（社員に占める比率が多い）

In [ ]:
# 評価（仕事への関与）別の退職者数

df_jb = pd.crosstab(df['Attrition'], df['JobInvolvement'])
print(df_jb)
df_jb.plot.bar()
# 3.0=Highでも辞めている（社員に占める比率が多い）

In [ ]:
# 評価（パフォーマンス）別の退職者数

df_pr = pd.crosstab(df['Attrition'], df['PerformanceRating'])
print(df_pr)
df_pr.plot.bar()

In [ ]:
# PerformanceRatingは３と４しかない？のね・・・すごい会社

df.groupby(['PerformanceRating'])['EmployeeID'].count()

量的カテゴリーデータを分析

In [ ]:
sns.boxplot(x="Attrition", y="Age", data=df)

In [ ]:
# 労働時間と退職
sns.boxplot(x="Attrition", y="time_std", data=df)

In [ ]:
# 月給と退職
sns.boxplot(x="Attrition", y="MonthlyIncome", data=df)

In [ ]:
# 勤続年数と退職
sns.boxplot(x="Attrition", y="TotalWorkingYears", data=df)

In [ ]:
# 給与UP率と退職
sns.boxplot(x="Attrition", y="PercentSalaryHike", data=df)

Attrition 辞めた人と辞めてない人の平均を比較する

In [ ]:
df_att = df.loc[df.Attrition =="Yes"]
df_att.head()

In [ ]:
df_natt = df.loc[df.Attrition =="No"]
df_natt.head()

In [ ]:
print(df_att.mean())
print(df_natt.mean())

In [ ]:
df_s = df_natt.mean() - df_att.mean()
df_s

In [ ]:
df_d = df_natt.mean() / df_att.mean()
df_d

差が大きい変数＝辞めない人の特徴

1. 今のMGRとの年数が長い：YearsWithCurrManager       1.531173
2. 全仕事年数が長い：TotalWorkingYears          1.436749
3. 在社年数が長い：YearsAtCompany             1.436231
4. 過去の昇進からの年数が長い：YearsSinceLastPromotion    1.148698
5. 仕事の満足度高い：JobSatisfaction            1.124500
6. 環境への満足度高い：EnvironmentSatisfaction    1.122995
7. 年齢高い：Age                        1.117641
8. 働いた社数少ない：NumCompaniesWorked         0.901963
9. 労働時間が少ない：time_std                   0.919471

2,3,8は同じ事かな

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['Attrition']=le.fit_transform(df['Attrition'])
df['BusinessTravel'] = le.fit_transform(df['BusinessTravel'])
df['Department'] = le.fit_transform(df['Department'])
df['EducationField'] = le.fit_transform(df['EducationField'])
df['Gender'] = le.fit_transform(df['Gender'])
df['JobRole'] = le.fit_transform(df['JobRole'])
df['MaritalStatus'] = le.fit_transform(df['MaritalStatus'])
df['Over18'] = le.fit_transform(df['Over18'])

In [ ]:
corr_cols = df[['Age','Attrition','BusinessTravel','DistanceFromHome',
                'Education', 'EducationField','Gender', 'JobLevel', 'JobRole',
                'MaritalStatus', 'MonthlyIncome', 'NumCompaniesWorked',
                'PercentSalaryHike', 'StockOptionLevel', 'TotalWorkingYears',
                'TrainingTimesLastYear', 'YearsAtCompany', 'YearsSinceLastPromotion',
                'YearsWithCurrManager','time_std', 'EnvironmentSatisfaction',
                'JobSatisfaction', 'WorkLifeBalance','JobInvolvement', 
                'PerformanceRating']]

In [ ]:
corr = corr_cols.corr()
plt.figure(figsize=(20,15))
sns.heatmap(corr, annot = True)
plt.show()

退職（Attrition）に相関している変数

1. time_std
2. TotalWorkingYears
3. MaritalStatus
4. YearsWithCurrManager
5. Age
6. YearsAtCompany
7. EnvironmentSatisfaction
8. JobSatisfaction

影響の大きい変数でロジスティック回帰モデル作成

In [ ]:
col = df[['time_std', 'TotalWorkingYears', 'MaritalStatus', 'YearsWithCurrManager',
      'Age','YearsAtCompany', 'EnvironmentSatisfaction', 'JobSatisfaction']]

In [ ]:
col.isnull().sum()

In [ ]:
col.fillna(0,inplace =True)

In [ ]:
col.isnull().any()

In [ ]:
X = col
y = df['Attrition']

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [ ]:
X_train,X_test, y_train, y_test = train_test_split(X,y, test_size = 0.20, random_state=50)

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
lr = LogisticRegression()
lr.fit(X_train,y_train)
y_pred = lr.predict(X_test)

print(accuracy_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

In [ ]:
print(classification_report(y_test,y_pred))

評価と報酬をみてみる

In [ ]:
# 労働時間と給与UPに退職の相関があるのか？　（28：time_std, 16：PercentSalaryHike）

plt.scatter(df.iloc[:, 28], df.iloc[:, 16], c=df['Attrition'])

In [ ]:
# 労働時間と月給に退職の相関があるのか？　（28：time_std, 1３：MonthlyIncome）

plt.scatter(df.iloc[:, 28], df.iloc[:, 13], c=df['Attrition'])
# 労働時間が長くて月給が低い層に退職（黄色）は多い

In [ ]:
# 年齢と月収に退職の相関があるのか？　（0：Age, 1３：MonthlyIncome）

plt.scatter(df.iloc[:, 0], df.iloc[:, 13], c=df['Attrition'])
# 給与は年齢では決まってないが、若手で月収低い層に退職が多い
# 給与が高い層、高年齢層でも退職は出現

In [ ]:
# 新たな変数を作成＝年齢と月給

df['ageincome'] = df['Age'] * df['MonthlyIncome'] /100

In [ ]:
# 労働時間と新変数（年齢＊月給）に退職の相関があるのか？　（28：time_std, ３４：ageincome）

plt.figure(figsize=(15,10))
plt.scatter(df.iloc[:, 28], df.iloc[:, 34], c=df['Attrition'])
# 労働時間２０００超えかつ、新変数２００００以下で退職が多い

In [ ]:
# 部門ごとの労働時間

sns.boxplot(x="Department", y="time_std",data=df)

In [ ]:
# パフォーマンス評価と給与UP率

sns.boxplot(x="PerformanceRating", y="PercentSalaryHike",data=df)

In [ ]:
# パフォーマンス評価と月給

sns.boxplot(x="PerformanceRating", y="MonthlyIncome",data=df)

# 評価は月収に反映されていない？？？

In [ ]:
# パフォーマンス評価と年齢

sns.boxplot(x="PerformanceRating", y="Age",data=df)

＜退職者分析＞
* 労働時間が長く月給が低い層が辞めている　
* 閾値はtime_std:2000, ageincome:20000
* 若くて独身者が辞めている
* 比較的社歴が浅く、今のマネジャーと短い期間の人が辞めている
* 満足度も一定現れる
* 評価は給与UPには反映されているようだが、月収は変わらない
* 時間当たりの待遇に不満を持っている可能性あり

＜対策提案＞
* 時間と給与のバランスを見直す
* 若くて独身者へのリテンション、特にマネジャーの関与
* タイムマネジメントでtime_std:2000を少なくする
* time_std:2000該当者、予備軍へのマネジメントフォロー強化
* 給与制度の見直し、残業手当など労働時間への対価を手厚くする